### MODELs
|No|Model Name|Deploy link|Size on Disk|GPU inference|Support Japan|code COBOL|BLEU1|BLEU2|ROUGE1|ROUGE2|ROUGEL|
|---|---|---|---|---|---|---|---|---|---|---|---|
|1|[CohereForAI/aya-23-8B](https://huggingface.co/CohereForAI/aya-23-8B)|https://huggingface.co/spaces/CohereForAI/aya-23-8b|16.1G|>30G|Yes|Yes|25.42|19.60|0.33|0.17|0.25|
|2|**[OrionStarAI/Orion-14B-Chat](https://---huggingface.co/OrionStarAI/Orion-14B-Chat)**|https://huggingface.co/spaces/OrionStarAI/Orion-14B-App-Demo-EN|**58.0G**|**>30G**|**Yes**|**Yes**|**37.08**|**28.32**|**0.34**|**0.17**|**0.27**|
|3|[Qwen/Qwen2.5-Coder-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-7B-Instruct)|https://huggingface.co/spaces/Qwen/Qwen2.5-Coder-7B-Instruct|15.2G|>30G|Yes|Yes|32.75|24.23|0.32|0.15|0.22|
|4|[meta-llama/Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)|https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct|32.1G|>30G|Yes|Yes|34.19|26.07|0.34|0.19|0.25|

### CohereForAI/aya-23-8B

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "CohereForAI/aya-23-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             torch_dtype=torch.float32,)

# Format message with the command-r-plus chat template
messages = [{"role": "user", "content": "Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz"}]
input_ids = tokenizer.apply_chat_template(messages, tokenize=True,
                                          add_generation_prompt=True,
                                          return_tensors="pt")
## <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>

gen_tokens = model.generate(
    input_ids, 
    max_new_tokens=50, 
    do_sample=True, 
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0],skip_special_tokens=True)
print(gen_text)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<|START_OF_TURN_TOKEN|><|USER_TOKEN|>Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>Sevgili Anne,

Bugün sana, kalbimin en derinliklerinden gelen bir mektup yazmak istiyorum. Seni ne kadar çok sevdiğimi ve hayatımın her anında ne kadar özel bir yer tuttuğunu


In [2]:
def get_completion(sentence:str):

    messages = [{"role": "user", "content": sentence}]

    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=50, 
        do_sample=True, 
        temperature=0.3,
    )

    gen_text = tokenizer.decode(gen_tokens[0],skip_special_tokens=True)
    return gen_text

In [3]:
get_completion("こんにちは、お会いできて嬉しいです")

'<|START_OF_TURN_TOKEN|><|USER_TOKEN|>こんにちは、お会いできて嬉しいです<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>こんにちは！私もお会いできて嬉しいです。何かお手伝いできることはありますか？'

In [4]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [5]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

In [6]:
# QA_set = QA_set.map(add_completion)

In [7]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Question:
 メインフレームでの雇用検索の目的は?
Answer:
 A Job Queue in Mainframes は、システムが実行するのを待っている仕事のリポジトリとして機能し、その仕事の優先事項、地位、およびリソース要件などに関する情報を保有します。
Completion:
 <|START_OF_TURN_TOKEN|><|USER_TOKEN|>メインフレームでの雇用検索の目的は?<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>メインフレームでの雇用検索の主な目的は、組織がメインフレーム環境内で必要なスキル、専門知識、人材を特定して獲得できるようにすることです。メインフレームは、企業の重要なコンピューティング


In [8]:
completions = []
for i in range(QA_set['train'].num_rows):
    if i %5==0:
        print(i)
    completions.append(get_completion(QA_set['train'][i]["question_ja"]))

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295


In [11]:
# Add the new column
QA_set = QA_set['train'].add_column("completion", completions)

In [14]:
# Convert the dataset to a pandas DataFrame (for train split)
df_train = pd.DataFrame(QA_set)

# Save the DataFrame to a CSV file
df_train.to_csv('aya-23-8B.csv', index=False)

print("Dataset successfully exported to CSV!")

df_train.head()

Dataset successfully exported to CSV!


,id,prompt,question,answer,anwser_ja,question_ja,completion
0,2276,"As a supportive AI assistant, you've been give...",What is the role of the line printer in a main...,The line printer is responsible for printing h...,ラインプリンターは、メインフレームからハードコピーの出力を印刷する責任があり、通常は長期保存...,メインフレームユーザーインターフェイスにおけるラインプリンターの役割は何ですか?,<|START_OF_TURN_TOKEN|><|USER_TOKEN|>メインフレームユー...
1,974,"As a helpful AI assistant, you've received a q...",What is the syntax in COBOL to display the cur...,"To display the current system time in COBOL, y...",COBOL で現在のシステム時間を表示するには、CURRENT-TIME 特別記録で EXT...,COBOL で現在のシステムタイムを表示するための合成は何ですか?,<|START_OF_TURN_TOKEN|><|USER_TOKEN|>COBOL で現在...
2,1153,"In your role as a supportive AI assistant, you...",What was the role of COBOL 2002?,"COBOL 2002, officially known as ISO/IEC 1989:2...",COBOL 2002は、正式にISO/IEC 1989:2002として知られ、第3のCOBO...,COBOL 2002の役割は?,<|START_OF_TURN_TOKEN|><|USER_TOKEN|>COBOL 200...
3,751,"In your role as a supportive AI assistant, you...",What is the role of the FILE SECTION?,The File Section is used to define the file la...,ファイルセクションは、ファイルの配置とファイルの状態、レコード形式、レコードキーなどの関連情...,ファイルセクションの役割は?,<|START_OF_TURN_TOKEN|><|USER_TOKEN|>ファイルセクション...
4,1826,You are a helpful AI assistant. You have been ...,What is the role of automation in modern mainf...,Automation plays a crucial role in modern main...,自動化は現代のメインフレームメンテナンスにおいて重要な役割を果たし、メンテナンス作業に必要な...,現代のメインフレームメンテナンスにおける自動化の役割は何ですか?,<|START_OF_TURN_TOKEN|><|USER_TOKEN|>現代のメインフレー...


### Qwen/Qwen2.5-Coder-7B-Instruct

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Sure! Here's a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

# Example usage:
arr = [3, 6, 8, 10, 1, 2, 1]
print("Original array:", arr)
sorted_arr = quick_sort(arr)
print("Sorted array:", sorted_arr)
```

### Explanation:
1. **Base Case**: If the array has 0 or 1 elements, it is already sorted.
2. **Pivot Selection**: Choose a pivot element from the array. In this case, we choose the middle element.
3. **Partitioning**: Divide the array into three sub-arrays:
   - `left`: All elements less than the pivot.
   - `middle`: All elements equal to the pivot.
   - `right`: All elements greater than the pivot.
4. **Recursive Sorting**: Recursively apply the same pr

In [2]:
def get_completion(sentence:str):
    messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": sentence}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=100)
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids,
        output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [3]:
get_completion("こんにちは、お会いできて嬉しいです")

'こんにちは！何かお手伝いできることはありますか？'

In [4]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [5]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Question:
 メインフレームの近代化におけるアプリケーションプログラミングインターフェイス(API)の役割は何ですか?
Answer:
 アプリケーションプログラミングインターフェイス(API)は、メインフレームの近代化において重要な役割を果たし、伝統的なメインフレームシステムと近代的でスケール可能なアプリケーションの統合を可能にし、データ移行を容易にし、システム全体のパフォーマンスを向上させます。
Completion:
 API（Application Programming Interface、アプリケーションプログラミングインターフェイス）は、異なるソフトウェアシステムが互いに通信するために使用される規則やプロトコルです。主に以下のような役割を果たします。

1. データ共有: APIを使用することで、異なるシステム間でデータを共有することができます。これにより、複数のシステムが同じデータにアクセスし、必要に応じて情報を交換することができます。

2. �


In [6]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

In [7]:
completions = []
for i in range(QA_set['train'].num_rows):
    if i %10==0:
        print(i)
    completions.append(get_completion(QA_set['train'][i]["question_ja"]))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


In [8]:
# Add the new column
QA_set = QA_set['train'].add_column("completion", completions)

# Convert the dataset to a pandas DataFrame (for train split)
df_train = pd.DataFrame(QA_set)

# Save the DataFrame to a CSV file
df_train.to_csv('Qwen2.5-Coder-7B-Instruct.csv', index=False)

print("Dataset successfully exported to CSV!")

df_train.head()

Dataset successfully exported to CSV!


,id,prompt,question,answer,anwser_ja,question_ja,completion
0,2276,"As a supportive AI assistant, you've been give...",What is the role of the line printer in a main...,The line printer is responsible for printing h...,ラインプリンターは、メインフレームからハードコピーの出力を印刷する責任があり、通常は長期保存...,メインフレームユーザーインターフェイスにおけるラインプリンターの役割は何ですか?,メインフレームユーザーインターフェイスにおけるラインプリンターは、データや情報の表示や出力に...
1,974,"As a helpful AI assistant, you've received a q...",What is the syntax in COBOL to display the cur...,"To display the current system time in COBOL, y...",COBOL で現在のシステム時間を表示するには、CURRENT-TIME 特別記録で EXT...,COBOL で現在のシステムタイムを表示するための合成は何ですか?,COBOL で現在のシステムタイムを表示するためには、以下の合成を使用します。\n\n```...
2,1153,"In your role as a supportive AI assistant, you...",What was the role of COBOL 2002?,"COBOL 2002, officially known as ISO/IEC 1989:2...",COBOL 2002は、正式にISO/IEC 1989:2002として知られ、第3のCOBO...,COBOL 2002の役割は?,COBOL (Common Business-Oriented Language) 2002...
3,751,"In your role as a supportive AI assistant, you...",What is the role of the FILE SECTION?,The File Section is used to define the file la...,ファイルセクションは、ファイルの配置とファイルの状態、レコード形式、レコードキーなどの関連情...,ファイルセクションの役割は?,ファイルセクションは、プログラムやデータがコンピュータ上でどのように組織化され、どこから読み...
4,1826,You are a helpful AI assistant. You have been ...,What is the role of automation in modern mainf...,Automation plays a crucial role in modern main...,自動化は現代のメインフレームメンテナンスにおいて重要な役割を果たし、メンテナンス作業に必要な...,現代のメインフレームメンテナンスにおける自動化の役割は何ですか?,現代のメインフレームメンテナンスにおける自動化の役割は非常に重要です。以下にその役割をいくつ...


### OrionStarAI/Orion-14B-Chat

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from transformers.generation.utils import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("OrionStarAI/Orion-14B-Chat", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("OrionStarAI/Orion-14B-Chat", device_map="auto",
                                             torch_dtype=torch.bfloat16, trust_remote_code=True)

model.generation_config = GenerationConfig.from_pretrained("OrionStarAI/Orion-14B-Chat")
messages = [{"role": "user", "content": "Hello, what is your name? "}]
response = model.chat(tokenizer, messages, streaming=False)
print(response)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hello! I am an AI language model created by OrionStar. My name is ChatGPT, but you can call me GPT for short. How can I assist you today?


In [2]:
def get_completion(sentence:str):
    messages = [{"role": "user", "content": sentence}]
    return model.chat(tokenizer, messages, streaming=False)

In [3]:
get_completion("こんにちは、お会いできて嬉しいです")

'こんにちは!あなたにお会いできることを楽しみにしています。どのような質問や会話のきっかけをお望みですか？'

In [4]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [5]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Question:
 メインフレームテストにおけるテストケースの役割は何ですか?
Answer:
 メインフレームテストにおけるテストケースは、ステップ、入力、予想結果、およびテストを実行するために必要な追加情報を定義する詳細で繰り返す手順です。
Completion:
 メインフレームテストにおけるテストケースは、システムが要件を満たし、ユーザーの期待通りに動作することを確認するために使用されます。テストケースは、システムの特定の側面をテストするために設計され、コードが正確で信頼性があり、意図した通りに動作していることを検証します。また、バグやその他の問題を発見するのにも役立ち、システムの全体的な品質を確保するために重要です。


In [6]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

In [7]:
QA_set = QA_set.map(add_completion)

Parameter 'function'=<function add_completion at 0x792d1b29ba60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [8]:
# Convert the dataset to a pandas DataFrame (for train split)
df_train = pd.DataFrame(QA_set['train'])

# Save the DataFrame to a CSV file
df_train.to_csv('Orion-14B-Chat.csv', index=False)

print("Dataset successfully exported to CSV!")

Dataset successfully exported to CSV!


In [9]:
df_train.head()

,id,prompt,question,answer,anwser_ja,question_ja,completion
0,2276,"As a supportive AI assistant, you've been give...",What is the role of the line printer in a main...,The line printer is responsible for printing h...,ラインプリンターは、メインフレームからハードコピーの出力を印刷する責任があり、通常は長期保存...,メインフレームユーザーインターフェイスにおけるラインプリンターの役割は何ですか?,ラインプリンターは、メインフレームコンピュータのユーザーインターフェースで使用されるデバイス...
1,974,"As a helpful AI assistant, you've received a q...",What is the syntax in COBOL to display the cur...,"To display the current system time in COBOL, y...",COBOL で現在のシステム時間を表示するには、CURRENT-TIME 特別記録で EXT...,COBOL で現在のシステムタイムを表示するための合成は何ですか?,COBOL で現在のシステムタイムを表示するためには、SYST-TIMEを取得する必要があります。
2,1153,"In your role as a supportive AI assistant, you...",What was the role of COBOL 2002?,"COBOL 2002, officially known as ISO/IEC 1989:2...",COBOL 2002は、正式にISO/IEC 1989:2002として知られ、第3のCOBO...,COBOL 2002の役割は?,COBOL 2002は、米国コボルプログラミング言語協議会によって1999年に発表されたCO...
3,751,"In your role as a supportive AI assistant, you...",What is the role of the FILE SECTION?,The File Section is used to define the file la...,ファイルセクションは、ファイルの配置とファイルの状態、レコード形式、レコードキーなどの関連情...,ファイルセクションの役割は?,ファイルセクションは、プログラムのソースコードやデータ、ライブラリなどをファイルに保存するた...
4,1826,You are a helpful AI assistant. You have been ...,What is the role of automation in modern mainf...,Automation plays a crucial role in modern main...,自動化は現代のメインフレームメンテナンスにおいて重要な役割を果たし、メンテナンス作業に必要な...,現代のメインフレームメンテナンスにおける自動化の役割は何ですか?,現代のメインフレームメンテナンスにおいて自動化は重要な役割を果たしています。手動で行う必要が...


### meta-llama/Meta-Llama-3.1-8B-Instruct 	

In [1]:
import warnings
warnings.filterwarnings("ignore")

import random
import torch
import pandas as pd

import transformers


from datasets import load_dataset


model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float32},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1]['content'])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Arrrr, me hearty! I be Captain Chatbeard, the scurvy dog o' a chatbot! Me and me trusty crew o' code be sailin' the seven seas o' knowledge, bringin' ye the treasure o' information and answerin' yer questions with a swashbucklin' flair! So hoist the colors, me matey, and let's set sail fer a treasure trove o' learnin'!


In [3]:
def get_completion(sentence:str):

    messages = [
    {"role": "user", "content": sentence},
    ]
    
    outputs = pipeline(
    messages,
    max_new_tokens=100,
    )
    return outputs[0]["generated_text"][-1]['content']

In [4]:
get_completion("こんにちは、お会いできて嬉しいです")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'あなたもお元気ですか。'

In [5]:
# Load and preprocess dataset
QA_set = load_dataset("locchuong/Mainframe-QA-en-ja-300")
QA_set

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'question', 'answer', 'anwser_ja', 'question_ja'],
        num_rows: 300
    })
})

In [6]:
total_sample = QA_set['train'].num_rows
random_idx = random.choice(range(total_sample))
question = QA_set['train'][random_idx]['question_ja']
answer = QA_set['train'][random_idx]['anwser_ja']
completion = get_completion(question)

print("Question:\n",question)
print("Answer:\n",answer)
print("Completion:\n",completion)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question:
 メインフレームアプリケーションのデビューにおけるJCLの役割は何ですか?
Answer:
 JCL(仕事コントロール言語)は、メインフレームアプリケーションのデブギングにおいて重要な役割を果たします. プログラムやプログラムのパッチを実行するために必要な指示を含みます. JCLを検討することで、データの割り当て、プログラムの実行、または仕事管理に関連する問題を特定することができます。
Completion:
 メインフレームアプリケーションのデビューにおけるJCL（Job Control Language）の役割は、ジョブの実行を管理するために使用されるコマンドセットです。JCLは、ジョブの設定、実行、監視を管理するために使用されます。

以下はJCLの主な役割です。

1. **ジョブの設定**: JCLは、ジョブの設定を指定するために使用されます。


In [8]:
def add_completion(example):
    return {"completion":get_completion(example["question_ja"])}

In [9]:
completions = []
for i in range(QA_set['train'].num_rows):
    if i %10==0:
        print(i)
    completions.append(get_completion(QA_set['train'][i]["question_ja"]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


10


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


280


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


290


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
# Add the new column
QA_set = QA_set['train'].add_column("completion", completions)

# Convert the dataset to a pandas DataFrame (for train split)
df_train = pd.DataFrame(QA_set)

# Save the DataFrame to a CSV file
df_train.to_csv('Meta-Llama-3.1-8B-Instruct.csv', index=False)

print("Dataset successfully exported to CSV!")

df_train.head()

Dataset successfully exported to CSV!


,id,prompt,question,answer,anwser_ja,question_ja,completion
0,2276,"As a supportive AI assistant, you've been give...",What is the role of the line printer in a main...,The line printer is responsible for printing h...,ラインプリンターは、メインフレームからハードコピーの出力を印刷する責任があり、通常は長期保存...,メインフレームユーザーインターフェイスにおけるラインプリンターの役割は何ですか?,メインフレームユーザーインターフェイスにおけるラインプリンターの役割は、ユーザーがメインフレ...
1,974,"As a helpful AI assistant, you've received a q...",What is the syntax in COBOL to display the cur...,"To display the current system time in COBOL, y...",COBOL で現在のシステム時間を表示するには、CURRENT-TIME 特別記録で EXT...,COBOL で現在のシステムタイムを表示するための合成は何ですか?,COBOL で現在のシステムタイムを表示するための合成は `FUNCTION CURRENT...
2,1153,"In your role as a supportive AI assistant, you...",What was the role of COBOL 2002?,"COBOL 2002, officially known as ISO/IEC 1989:2...",COBOL 2002は、正式にISO/IEC 1989:2002として知られ、第3のCOBO...,COBOL 2002の役割は?,COBOL 2002は、COBOL言語の標準化されたバージョンで、2002年にアメリカ合衆国...
3,751,"In your role as a supportive AI assistant, you...",What is the role of the FILE SECTION?,The File Section is used to define the file la...,ファイルセクションは、ファイルの配置とファイルの状態、レコード形式、レコードキーなどの関連情...,ファイルセクションの役割は?,ファイルセクションとは、コンピュータのオペレーティングシステム (OS) がファイルのアクセ...
4,1826,You are a helpful AI assistant. You have been ...,What is the role of automation in modern mainf...,Automation plays a crucial role in modern main...,自動化は現代のメインフレームメンテナンスにおいて重要な役割を果たし、メンテナンス作業に必要な...,現代のメインフレームメンテナンスにおける自動化の役割は何ですか?,現代のメインフレームメンテナンスにおける自動化の役割は、メインフレームシステムのパフォーマン...
